# Визуализация

В модуле собраны скрипты для визуализации данных. В основном они полезны для анализа задачи бинарной классификации.

Для начала сгенерируем игрушечный пример.

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd
import numpy as np

np.random.seed(42)

N=10000

X, y = make_classification(n_features=4, n_samples=N)

df = pd.DataFrame(X, columns=['feature_%d' % i for i in range(X.shape[1])])
df['y'] = y
df['sample_date'] = np.random.choice(pd.date_range('2025-01-01', '2025-12-31'), N)
df['category_feature'] = np.random.choice(['foo', 'bar', np.nan], N)

df.head(2)

В игрушечной выборке содержатся:

*  несколько признаков `feature_*`
*  бинарная целевая переменная - `y`
*  поле с датой - `sample_date`
*  категорияльное поле `category_feature`

Для работы скриптов потребуется пакет [holoviews](http://holoviews.org). Большинство скриптов - функции, возвращающие одну картинку, а точнее [Overlay](http://holoviews.org/reference/containers/bokeh/Overlay.html) с различными точками, линимями, ...  

In [ ]:
import holoviews as hv
hv.extension('matplotlib')

Проилюстируем работу некоторых функций

In [ ]:
from risksutils.visualization import woe_line, woe_stab, cross_tab, isotonic, distribution

### woe_line

In [ ]:
woe_line(df=df, feature='feature_2', target='y', num_buck=30)

Данная функция разбивает числовой признак `feature` на `num_buck` бакетов. И в каждом бакете считается <br>
$\text{Weight of Evidence} = \ln\left(\frac{\text{доля 1 в бакте}}{\text{доля 0 в бакете}}\right)
- \ln\left(\frac{\text{доля 1 во всей выборке}}{\text{доля 0 во всей выборке}}\right)$.  

Если в бакете доля объектов класса 1 совпадает с долей 1 во всей выборке, то $WoE = 0$. Если в бакете присутствуют только объекты одного класса, то WoE будет равно бесконечности – из-за взятия логарифма. В данной функции доля объектов каждого класса ограничивается 0.001 - снизу и 0.999 - сверху.

На самом графике [woe_line](#woe_line) показана зависимость WoE в бакете от среднего значения признака `feature` в нем. 

На примере графика выше можно сказать, что среди объектов со значением feature_2 > 1 гораздо чаще присутствует 1.

### isotonic

In [ ]:
isotonic(df=df, predict='feature_2', target='y')

График [isotonic](#isotonic) похож на график [woe_line](#woe_line) - так же отображается зависимость частоты объектов класса 1 от значений признака, только явная разбивка на бакеты отсутствует. Построение зависимости основано на [Isotonic Regression](http://scikit-learn.org/stable/auto_examples/plot_isotonic_regression.html), которая восстанавливает монотонную зависимость. 

Использовать [isotonic](#isotonic) совместно с доверительными интервалами удобно для проверки совпадения прогноза с фактическими данными. Так, как обычно, предполагается монотонное влияние прогноза на целевую переменную.

### distribution

In [ ]:
distribution(df=df, feature='feature_2', date='sample_date', num_buck=4)

Данная диаграмма отражает изменение распределений признака `feature` во времени `date`. Признак дискретезируется разбивкой на бакеты. Затем выборка разбивается на группы (по умолчанию на месяца – параметр `date_freq`), и в каждой группе считается доля объектов из каждого бакета. По данному графику [distribution](#distribution) удобно обнаруживать изменения в расчете признака во времени.

### woe_stab

In [ ]:
woe_stab(df=df, feature='feature_2', target='y', date='sample_date', num_buck=3)

На данном графике отображается изменение влияния признака `feature` на целевую переменную `target` во времени `date`. Для этого признак разбивается на бакеты и для каждой временной группы считаются значения WoE.

В данном игрушечном примере видно, что влияние `feature_2` на `y` стабильно по времени. Это и должно быть, так как мы сгенерировали поле `sample_date` случайно и независимо от остальной выборки.

### cross_tab

In [ ]:
cross_tab(df, 'feature_2', 'category_feature', 'y', num_buck1=3, num_buck2=3)

Данный скрипт отличается тем, что возвращает не объект [holoviews](http://holoviews.org), а набор [pandas.dataframe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html), а точнее набор [Styler](https://pandas.pydata.org/pandas-docs/stable/style.html) - dataframe c визуальными настройками. 

В [cross_tab](#cross_tab) визуализируется совместное влияние пары признаков на целевую переменную (аналогично [pandas.crosstab](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html)). Каждый признак разбивается на бакеты и считается доля объектов класса 1 в каждой комбинации пары бакетов - первая таблица. А так же считается общее количество объектов - вторая таблица. Вместе с этим выводятся агрегированные статистики - последние строка и столбец.